<a href="https://colab.research.google.com/github/yzbaya/FAD.Machine-Learning/blob/main/FAD_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [21]:
data = pd.read_csv('historique ventes.csv', encoding='ISO-8859-1')

<ipython-input-21-7b4516ff9bda>:1: DtypeWarning: Columns (5,14,22,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('historique ventes.csv', encoding='ISO-8859-1')


**Préparation des Données **

In [ ]:
#Nettoyage des données DDV DDA
# data['DDA'] = pd.to_datetime(data['DDA'], errors='coerce', dayfirst=True)
# data['DDV'] = pd.to_datetime(data['DDV'], errors='coerce', dayfirst=True)
# data['month_dernier_vente']=data['DDV'].dt.month
# data['season_dernier_vente']=data['DDV'].dt.month % 12 // 3+1

In [26]:
numeric_features = ['NJR','STDA','Ecoulement']
# columns_to_convert = ['STDA', 'CMJ', 'Ecoulement']
print(data[numeric_features].dtypes)


NJR           float64
STDA            int64
Ecoulement    float64
dtype: object


In [34]:
from sklearn.impute import SimpleImputer
#extract et imputer les colonnes numeriques

numeric_features = ['NJR','CMJ','STDA','Ecoulement']
print(' colonnes numeriques')
print(data[numeric_features].dtypes)
imputer_numeric=SimpleImputer(strategy='median')
data[numeric_features]=imputer_numeric.fit_transform(data[numeric_features])

# List des colonnes à convertir et à imputer
columns_to_convert = ['Vente 2022','Vente 2023']
print('autres colonnes')
print(data[columns_to_convert].dtypes)
def convert_data(data,columns_to_convert):
  for column in columns_to_convert:
    data[column]=data[column].replace({',':''},regex=True)
    data[column]=pd.to_numeric(data[column],errors='coerce')
    imputer_cols=SimpleImputer(strategy='median')
    data[column]=imputer_cols.fit_transform(data[[column]])
convert_data(data,columns_to_convert)
#test
print('colonnes non num')
print(data[columns_to_convert].dtypes)

print("Vérification générale des valeurs manquantes:")
print(data[columns_to_convert].isna().sum())

 colonnes numeriques
NJR           float64
CMJ           float64
STDA          float64
Ecoulement    float64
dtype: object
autres colonnes
Vente 2022    float64
Vente 2023    float64
dtype: object
colonnes non num
Vente 2022    float64
Vente 2023    float64
dtype: object
Vérification générale des valeurs manquantes:
Vente 2022    0
Vente 2023    0
dtype: int64


In [ ]:
#sélection des données catégorial

categorical_features = ['Pareto']
# Imputation pour des valeurs categorial par la médiane
imputer_mostfrequent=SimpleImputer(strategy='most_frequent')
data[categorical_features] = imputer_mostfrequent.fit_transform(data[categorical_features])
 #encodage one_hot encoding
 data = pd.get_dummies(data,colums=['Pareto'],prefix='Pareto')



   NJR
0  250
1   70
2  178
3   50
4  177
  Pareto    CMJ STDA Ecoulement
0     DD  0,009    0    117,000
1     BB  0,926    0     46,440
2     DD  0,122   18    172,565
3     BB  2,271    1     20,253
4     CD  0,204    1     14,706


In [ ]:
from sklearn.model_selection import train_test_split
x=data[numeric_features + categorical_features]
# y=data[target]

KeyError: 'vente_future'